<a href="https://colab.research.google.com/github/Dogdriip/notebooks/blob/main/TextRank_newspaper_sentence_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install newspaper3k

     |████████████████████████████████| 215kB 3.2MB/s 
     |████████████████████████████████| 7.4MB 6.9MB/s 
     |████████████████████████████████| 51kB 5.4MB/s 
     |████████████████████████████████| 81kB 8.5MB/s 
  Created wheel for jieba3k: filename=jieba3k-0.35.1-cp36-none-any.whl size=7398406 sha256=583a6609bfc768280d5ba8eae72246d27d20bd5412aa86e1faed2c2329b9b391
  Stored in directory: /root/.cache/pip/wheels/83/15/9c/a3f1f67e7f7181170ad37d32e503c35da20627c013f438ed34
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-cp36-none-any.whl size=3355 sha256=002d8eb5cba09ec6130fb0857519a72bcafc9c0f22238db4a25969c359067a26
  Stored in directory: /root/.cache/pip/wheels/de/03/ca/778e3a7a627e3d98836cc890e7cb40c7575424cfd3340f40ed
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-cp36-none-any.whl size=13538 sha256=ccc8809ff5abf85d17075692c4673e7b854c131b997d9670b3328c8d866079ff
  Stored in directory: /root/.cache/pip/wheels/81/2b/43/a02ede72324dd40cdd7ca53aad718c771

In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 92kB 8.3MB/s 
     |████████████████████████████████| 3.8MB 46.0MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [ ]:
from newspaper import Article
from konlpy.tag import Kkma
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np

In [ ]:
kkma = Kkma()
print(kkma.pos("안녕하세요? 오늘 날씨가 참 좋네요."))

[('안녕', 'NNG'), ('하', 'XSV'), ('세요', 'EFN'), ('?', 'SF'), ('오늘', 'NNG'), ('날씨', 'NNG'), ('가', 'JKS'), ('참', 'MAG'), ('좋', 'VA'), ('네요', 'EFN'), ('.', 'SF')]


In [ ]:
class SentenceTokenizer(object):
  def __init(self):
    self.kkma = Kkma()
    self.twitter = Twitter()
    self.stopwords = ['중인' ,'만큼', '마찬가지', '꼬집었', "연합뉴스", "데일리", "동아일보", "중앙일보", "조선일보", "기자","아", "휴", "아이구", "아이쿠", "아이고", "어", "나", "우리", "저희", "따라", "의해", "을", "를", "에", "의", "가",]

  def url2sentences(self, url):
    article = Article(url, language="ko")
    article.download()
    article.parse()
    sentences = self.kkma.sentences(article.text)

    for idx in range(len(sentences)):
      if len(sentences[idx]) <= 10:  # if length of current sentence is lte 10, concat next sentence with current sentence.
        sentences[idx - 1] += (" " + sentences[idx])
        sentences[idx] = ""

    return sentences
  
  def get_nouns(Self, sentences):
    nouns = []
    for sentence in sentences:
      if sentence is not "":
        nouns.append(" ".join([noun for noun in self.twitter.nouns(str(sentence)) if noun not in self.stopwords and len(noun) > 1]))
    
    return nouns


In [ ]:
class GraphMatrix(object):
  def __init__(self):
    self.tfidf = TfidfVectorizer()
    self.cnt_vec = CountVectorizer()
    self.graph_sentence = []
  
  # 명사로 이루어진 문장을 입력받아 sklearn의 TfidfVectorizer.fit_transform을 이용하여 tfidf matrix를 만든 후 Sentence graph를 return 한다.
  def build_sentence_graph(self, sentence):
    tfitf_mat = self.tfidf.fit_transform(sentence).toarray()
    self.graph_sentence = np.matmul(tfidf_mat, tfidf_mat.T)
    return self.graph_sentence

  # 명사로 이루어진 문장을 입력받아 sklearn의 CountVectorizer.fit_transform을 이용하여 matrix를 만든 후 word graph와 {idx: word}형태의 dictionary를 return한다.
  def build_words_graph(self, sentence):
    cnt_vec_mat = normalize(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
    vocab = self.cnt_vec.vocabulary_
    return np.matmul(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}


In [ ]:
class Rank(object):
  def get_ranks(self, graph, d=0.85):
    A = graph
    matrix_size = A.shape[0]
    for id in rnage(matrix_size):
      A[id, id] = 0  # diagonal goes to zero
      link_sum = np.sum(A[:, id])
      if link_sum != 0:
        A[:, id] /= link_sum
      A[:, id] *= -d
      A[id, id] = 1

    B = (1 - d) * np.ones((matrix_size, 1))
    ranks = np.linalg.solve(A, B)  # 연립방정식 Ax = B
    return {idx: r[0] for idx, r in enumerate(ranks)}


In [ ]:
!pip list

Package                       Version        
----------------------------- ---------------
absl-py                       0.10.0         
alabaster                     0.7.12         
albumentations                0.1.12         
altair                        4.1.0          
argon2-cffi                   20.1.0         
asgiref                       3.2.10         
astor                         0.8.1          
astropy                       4.0.1.post1    
astunparse                    1.6.3          
async-generator               1.10           
atari-py                      0.2.6          
atomicwrites                  1.4.0          
attrs                         20.2.0         
audioread                     2.1.8          
autograd                      1.3            
Babel                         2.8.0          
backcall                      0.2.0          
beautifulsoup4                4.6.0          
bleach                        3.2.1          
blis                          0.4.